---------- Start of generic setup code to get kaggle credentials login and data to work with. Copied much from Workshop 2 ----------

In [1]:
# Code to read csv file into Colaboratory:
!pip install -U -q PyDrive
from pydrive.auth import GoogleAuth
from pydrive.drive import GoogleDrive
from google.colab import auth
from oauth2client.client import GoogleCredentials

In [2]:
# Authenticate and create the PyDrive client.
auth.authenticate_user()
gauth = GoogleAuth()
gauth.credentials = GoogleCredentials.get_application_default()
drive = GoogleDrive(gauth)

https://drive.google.com/file/d/1DeYgEzZ8Bs8Y3QsHd5YsV-3D-SzVAm4x

In [3]:
#GET data from file I've uploaded to my drive. Use your own kaggle.json and google auth logins etc
downloaded = drive.CreateFile({'id':'1DeYgEzZ8Bs8Y3QsHd5YsV-3D-SzVAm4x'}) 
downloaded.GetContentFile('predict-energy-consumption.zip')

In [4]:
!unzip predict-energy-consumption.zip

!ls

Archive:  predict-energy-consumption.zip
  inflating: sample_submission.csv   
  inflating: test.csv                
  inflating: train.csv               
predict-energy-consumption.zip	sample_submission.csv  train.csv
sample_data			test.csv


In [5]:
from google.colab import files

files.upload(); # upload your kaggle.json file

Saving kaggle.json to kaggle.json


Processing data from the source.

In [6]:
import json

!mkdir /root/.kaggle/
!mv kaggle.json /root/.kaggle/kaggle.json
!chmod 600 ~/.kaggle/kaggle.json
!kaggle config set -n path -v{/content}

- path is now set to: {/content}


In [ ]:
#GENERIC SUBMIT FUNCTION
!kaggle competitions submit -c predict-electricity-consumption -f sample_submission.csv -m "Sample submission test"

In [11]:
#ACTUAL CODING PART BELOW HERE:
import numpy as np 
import pandas as pd
 
# note that we use only 1M out of 2M records
train =  pd.read_csv('train.csv')
test = pd.read_csv('test.csv')

for col in train.columns:
  train[col] = train[col].replace(np.nan, 0)

for col in test.columns:
  test[col] = test[col].replace(np.nan, 0)

#print(train)
#print(test)

#At this point should have a train and test without Nan values:)

---------- End of generic setup code. Actual programming can begin after here ----------